# Lecture de données

In [9]:
#lecture du fichier
import pandas as pd
import numpy as np
base =  pd.read_csv("C:/Users/Cyrie/OneDrive/Bureau/M2_DS/S1/MLOPS/athlete_events.csv")  


# Filtre sur Summer

In [65]:
#filtre sur Summer
base_summer = base[base['Season'] == 'Summer']

#nombre de ligne de notre df base après filtre du df base summer
nombre_de_lignes1 = base_summer.shape[0]

#affiche le nombre de lignes du df base summer 2000
print("Nombre de lignes :", nombre_de_lignes1)

Nombre de lignes : 222552


# Valeurs manquantes et abbérantes du df base_summer

In [ ]:
#valeurs manquantes pour le df base summer
valeurs_manquantes_par_variable1 = base_summer.isnull().sum()

# Affichage du résultat pour le df base summer
print(valeurs_manquantes_par_variable1)

In [ ]:
#valeur abbérentes
# Création de la figure et des axes
plt.figure(figsize=(8, 6))

###################AGE#####################
# Création de la boîte à moustaches 
sns.boxplot(x=base_summer['Medal'], y=base_summer['Age'], color='#008030')

# Titres et labels
plt.title('Boîte à moustaches de la variable age selon la médaille')
plt.xlabel('Médaille')
plt.show()

###################Height####################
# Création de la boîte à moustaches 
sns.boxplot(x=base_summer['Medal'], y=base_summer['Height'], color='#008080')

# Titres et labels
plt.title('Boîte à moustaches de la variable taille selon la médaille')
plt.xlabel('Médaille')
plt.show()


###################Weight####################
# Création de la boîte à moustaches 
sns.boxplot(x=base_summer['Medal'], y=base_summer['Weight'], color='#008030')

# Titres et labels
plt.title('Boîte à moustaches de la variable poids selon la médaille')
plt.xlabel('Médaille')
plt.show()




In [ ]:
#lecture des valeurs abbérantes
# Filtre sur les enregistrements où l'âge est supérieur à 40 et inférieur à 11 ans
base_summer_age_11_40 = base_summer[(base_summer['Age'] > 40) | (base_summer['Age'] < 11)]
print("DataFrame pour l'âge entre 11 et 40 ans inclus:")
print(base_summer_age_11_40)

# Filtre sur les enregistrements où la taille est inférieure à 145 cm et supérieure à 210 cm
base_summer_height_145_210 = base_summer[(base_summer['Height'] > 210) | (base_summer['Height'] < 145)]
print("\nDataFrame pour la taille entre 145 et 210 cm inclus:")
print(base_summer_height_145_210)

# Filtre sur les enregistrements où le poids est inférieur à 35 kg et supérieur à 118 kg
base_summer_weight_35_118 = base_summer[(base_summer['Weight'] > 118) | (base_summer['Weight'] < 35)]
print("\nDataFrame pour le poids entre 35 et 118 kg inclus:")
print(base_summer_weight_35_118)


# Regroupements

In [ ]:
#voir la répartiton des variables quantitatives avant le regroupement
###################YEAR####################
plt.figure(figsize=(8, 6))
sns.histplot(base_summer['Year'], bins=30, kde=True, color='#008030', stat='density', element='bars', common_norm=False)  # Utilisation du code hexadécimal pour un bleu canard foncé
sns.kdeplot(base_summer['Year'], color='red', linewidth=2)  # Courbe KDE en rouge
plt.title('Répartition de la variable année ')
plt.xlabel('Année')
plt.ylabel('Densité')
plt.show()

###################AGE#####################
plt.figure(figsize=(8, 6))
sns.histplot(base_summer['Age'], bins=30, kde=True, color='#008080', stat='density', element='bars', common_norm=False)  # Utilisation du code hexadécimal pour un bleu canard foncé
sns.kdeplot(base_summer['Age'], color='red', linewidth=2)  # Courbe KDE en rouge
plt.title('Répartition de la variable age ')
plt.xlabel('Age')
plt.ylabel('Densité')
plt.show()

###################HEIGHT#####################
plt.figure(figsize=(8, 6))
sns.histplot(base_summer['Height'], bins=30, kde=True, color='#008030', stat='density', element='bars', common_norm=False)  # Utilisation du code hexadécimal pour un bleu canard foncé
sns.kdeplot(base_summer['Height'], color='red', linewidth=2)  # Courbe KDE en rouge
plt.title('Répartition de la variable taille ')
plt.xlabel('Taille')
plt.ylabel('Densité')
plt.show()

###################WEIGHT#####################
plt.figure(figsize=(8, 6))
sns.histplot(base_summer['Weight'], bins=30, kde=True, color='#008030', stat='density', element='bars', common_norm=False)  # Utilisation du code hexadécimal pour un bleu canard foncé
sns.kdeplot(base_summer['Weight'], color='red', linewidth=2)  # Courbe KDE en rouge
plt.title('Répartition de la variable poids ')
plt.xlabel('Poids')
plt.ylabel('Densité')
plt.show()

In [ ]:
#faire les regrouepements sur les variables quantitatives
###################AGE#####################
#intervalles d'age
intervalles = [0, 21, 25, 33, float('inf')]
labels = ['< 21 ans', '21 - 24 ans', '25-32 ans', '> 32 ans']
#nouvelle colonne "Classe_age" basée sur les intervalles définis
base_summer['Classe_age'] = pd.cut(base_summer['Age'], bins=intervalles, labels=labels, right=False)
#convertir la colonne "Classe_age" en catégorie si nécessaire
base_summer['Classe _age'] = base_summer['Classe_age'].astype('category')
#afficher les premières lignes du DataFrame avec la nouvelle colonne
print(base_summer[['Age', 'Classe_age']].head(5))

###################HEIGHT#####################
intervalles_taille = [float('-inf'), 165, 173, 186, float('inf')]
labels_taille = ['< 165 cm', '165 - 172 cm', '173 - 185 cm', '> 185 cm']
base_summer['Classe_height'] = pd.cut(base_summer['Height'], bins=intervalles_taille, labels=labels_taille, right=False)
base_summer['Classe_height'] = base_summer['Classe_height'].astype('category')
print(base_summer[['Height', 'Classe_height']].head(5))

###################WEIGHT#####################
intervalles_poids = [float('-inf'), 65, 74, 81, float('inf')]
labels_poids = ['< 65 kg', '65 - 73 kg', '74 - 80 kg', '> 80 kg']
base_summer['Classe_weight'] = pd.cut(base_summer['Weight'], bins=intervalles_poids, labels=labels_poids, right=False)
base_summer['Classe_weight'] = base_summer['Classe_weight'].astype('category')
print(base_summer[['Weight', 'Classe_weight']].head(70))


In [ ]:
#afficher les frequences de modalites pour les variables regroupes
###################AGE#####################
print("Fréquences de modalités pour la Classe âge:")
print(base_summer['Classe_age'].value_counts())

###################HEIGHT#####################
print("\nFréquences de modalités pour la Classe taille:")
print(base_summer['Classe_height'].value_counts())

###################WEIGHT#####################
print("\nFréquences de modalités pour la Classe poids:")
print(base_summer['Classe_weight'].value_counts())


#fonction pour afficher les fréquences en pourcentage
def print_percentage_counts(column_name):
    percentages = base_summer[column_name].value_counts(normalize=True) * 100
    print(f"Fréquences en pourcentage pour la colonne '{column_name}':")
    print(percentages)

# Afficher les fréquences en pourcentage pour la classe d'âge
print_percentage_counts('Classe_age')

# Afficher les fréquences en pourcentage pour la classe de taille
print("\n")
print_percentage_counts('Classe_height')

# Afficher les fréquences en pourcentage pour la classe de poids
print("\n")
print_percentage_counts('Classe_weight')


# Filtre les années 2000 et suppression de variable

In [69]:
#filtre sur l annee sup ou egal a 2000
base_summer_2000 = base_summer[base_summer['Year'] >= 2000]

#nombre de ligne de notre df base après filtre du df base summer 2000
nombre_de_lignes2 = base_summer_2000.shape[0]

#affiche le nombre de lignes du df base summer 2000
print("Nombre de lignes :", nombre_de_lignes2)

Nombre de lignes : 67474


In [ ]:
# Supprimer les variables 'Team' et 'Season' du DataFrame base_summer_2000
base_summer_2000 = base_summer_2000.drop(['Team', 'Season'], axis=1)

# Afficher les premières lignes du DataFrame après la suppression
print(base_summer_2000.head())

# Valeurs manquantes du df base_summer_2000

In [ ]:
#valeurs manquantes pour le df base summer 2000
valeurs_manquantes_par_variable2 = base_summer_2000.isnull().sum()

# Affichage du résultat pour le df base 
print(valeurs_manquantes_par_variable2)


# Statistiques descriptives

In [ ]:
#statistiques desc
import seaborn as sns
import matplotlib.pyplot as plt

def create_bar_plot(data, x_variable, target_variable='Medal'):

    # Create a bar plot
    plt.figure(figsize=(12, 6))
    sns.countplot(x=x_variable, hue=target_variable, data=data, palette='viridis')
    plt.title(f'Count of {target_variable} by {x_variable}')
    plt.xlabel(x_variable)
    plt.ylabel('Count')
    plt.xticks(rotation=45)
    plt.legend(title=target_variable)
    plt.show()

# Example usage:
# Assuming your DataFrame is named df
create_bar_plot(base_summer, 'Sex')
create_bar_plot(base_summer, 'Age')
create_bar_plot(base_summer, 'Height')
create_bar_plot(base_summer, 'Weight')
create_bar_plot(base_summer, 'NOC')
create_bar_plot(base_summer, 'Games')
create_bar_plot(base_summer, 'Year')
create_bar_plot(base_summer, 'City')
create_bar_plot(base_summer, 'Sport')
create_bar_plot(base_summer, 'Event')

